In [38]:
import numpy as np
import matplotlib.pyplot as plt

## Axes
The first thing we've got to decide for our world is the axis orientation.  Suppose the world is a rectangle of size (*w*,*h*).  Do we want the world to have the top-left corner be at (0,0), or do we want the top-left to be (0,*h*)? Let's see how it's being drawn in our environment in OpenGL. In OpenGL, as Y increases we move *up*.  So our origin of (0,0) is going to be in the bottom-left.

## Relative Direction
We need to be able to get a normalized vector pointing from a source point (such as the agent) towards another point (such as food).  That vector should be normalized, so that we can multiply it by a scalar velocity.

In [35]:
ant=(1,1) # lower left
ant_dir = np.linalg

In [4]:
np.linalg.norm(np.array([9,9]))

12.727922061357855

In [9]:
ant_looking_at = np.array([100,100])
ant_pos = np.array([1,1])
ant_delta = ant_looking_at - ant_pos
ant_norm = np.linalg.norm(ant_delta)
ant_dir = ant_delta / ant_norm
print('delta',ant_delta)
print('ant_norm',ant_norm)
print('ant_dir',ant_dir)

delta [99 99]
ant_norm 140.0071426749364
ant_dir [0.70710678 0.70710678]


In [10]:
def vec_dir(v0, v1):
    """Returns a normalized vector pointing from v0 towards v1"""
    delta = v1 - v0
    norm = np.linalg.norm(delta)
    return delta / norm

In [11]:
vec_dir(ant_pos, (100,100))

array([0.70710678, 0.70710678])

In [23]:
p0 = np.array((1,1))
for p in [(100,100), (70,2), (50,12), (16, 80)]:
    print('From %s to %s dir=%s' % (p0, p, vec_dir(p0, p)))

From [1 1] to (100, 100) dir=[0.70710678 0.70710678]
From [1 1] to (70, 2) dir=[0.999895   0.01449123]
From [1 1] to (50, 12) dir=[0.97571625 0.21903834]
From [1 1] to (16, 80) dir=[0.18654062 0.98244725]


In [ ]:
test_vec_dir()

## Left vs Right

Part of the state is a sensory signal telling the agent whether something (food or the nest) is to the left or right, relative to the current facing direction.  We could do this with a vector *cross product*, or we could do this with angle determination using *atan2*.

In [39]:
def angle_from(p0, p1):
    """Returns the angle from p0 to p1"""
    delta = p1 - p0
    x, y = delta
    return np.arctan2(y, x)

In [40]:
ant = np.array((1,1)) # lower-left
target = np.array((99,10)) # Right side and a little up
angle_from(ant, target) # Expect this to be a little to the left, or a slight positive angle

0.09157985014268566

In [41]:
ant = np.array((1,20)) # A little up the left side.
target = np.array((99,10)) # Lower-right
angle_from(ant, target) # Expect this to be a little to the right, or a slight negative angle

-0.10168885176307704

In [55]:
def to_deg(a):
    return a / np.pi * 180

In [57]:
to_deg(np.pi/4), to_deg(np.pi/2), to_deg(np.pi), to_deg(np.pi * 3 / 2)

(45.0, 90.0, 180.0, 270.0)

In [89]:
def relative_angle(p0, dir0, p1):
    """
    p0: A vector with the position of an object.
    dir0: A normalized vector indicating the direction the object at p0 is facing.
    p1: The position of a target object.
    Returns the relative angle left/right that point p1 lies in the field of view of p0.
    Returns a positive number if the target is to the left, and negative if it is to the right.
    """
    x0, y0 = dir0
    angle0 = np.arctan2(y0, x0)
    angle1 = angle_from(p0, p1)
    delta_angle = angle1 - angle0
    #print('relang p0=%s dir0=%s p1=%s' % (p0, dir0, p1))
    #print('angle0=%s angle1=%s delta=%s' % (to_deg(angle0), to_deg(angle1), to_deg(delta_angle)))
    return delta_angle

In [61]:
def test_relative_angle():
    p0 = np.array((1,1))
    p0_facing = np.array((20, 100)) # Looking a little to the right of the top left corner.
    p0_dir = vec_dir(p0, p0_facing) # Mostly up and a little right. [0.18847945 0.98207713]
    targets = [(100,100), (70,2), (50,12), (16, 80)]
    for t in targets:
        print('From %s to %s' % (p0, t))
        a = relative_angle(p0, p0_dir, t)
        print('t=%s a=%s' % (t, to_deg(a)))
test_relative_angle()

From [1 1] to (100, 100)
angle0=79.13594007791228 angle1=45.0 delta=-34.13594007791228
t=(100, 100) a=-34.13594007791228
From [1 1] to (70, 2)
angle0=79.13594007791228 angle1=0.8303154862580109 delta=-78.30562459165428
t=(70, 2) a=-78.30562459165428
From [1 1] to (50, 12)
angle0=79.13594007791228 angle1=12.65255650055797 delta=-66.48338357735432
t=(50, 12) a=-66.48338357735432
From [1 1] to (16, 80)
angle0=79.13594007791228 angle1=79.24903300681196 delta=0.11309292889967655
t=(16, 80) a=0.11309292889967655


In [92]:
def test_relative_angle2():
    def offset(pt, a, r): # Returns a point at the given angle and distance
        x0,y0 = pt
        x1 = r * np.cos(a) + x0
        y1 = r * np.sin(a) + y0
        return np.array((x1, y1))
    p0 = np.array([20,20])
    a = 0
    r = 5
    # For several points around a circle, select a target point and face that point.
    # Then place a target a little to the left of that.
    for i in range(8):
        p0_dir = vec_dir(p0, offset(p0, a, r))
        target_a = a + np.pi/12 # Target is a little to the left
        p1 = offset(p0, target_a, r)
        ra = relative_angle(p0, p0_dir, p1)
        assert (ra  - np.pi/12) < 0.0001, 'expected angle to be equal'
        a += np.pi / 8
test_relative_angle2()

In [72]:
-190 % 360

170

In [81]:
pt = np.array([1,2])
x,y = pt
x,y

(1, 2)

In [98]:
x, y = np.random.uniform(-1,1), np.random.uniform(-1, 1)
norm = np.linalg.norm((x,y))
np.array((x,y)) / norm

array([ 0.66581168, -0.74611984])

In [99]:
def random_dir():
    x, y = np.random.uniform(-1,1), np.random.uniform(-1, 1)
    pt = np.array((x,y))
    return pt / np.linalg.norm(pt)

In [109]:
random_dir(), np.linalg.norm(random_dir())

(array([0.94738055, 0.32010951]), 0.9999999999999999)

In [110]:
np.floor(1.7)

1.0